In [1]:
#Importer la bibliotheque findspark pour ajouter pyspark to sys.path at runtime, cela rend sparkContext disponible dans le code
#Importer la bibliotheque pyspark
import findspark
findspark.init()
import pyspark

In [2]:
#Point d'entrée principale pour utiliser les fonctionnalités de spark. SparkContext represente la connection à un cluster Spark.
from pyspark.context import SparkContext
#Point d'entrée pour interagir avec les Datasets et les DataFrames.
from pyspark.sql.session import SparkSession

In [3]:
#Recuperer une SparkSession ou bien crùer une nouvelle en se basant sur les options passées au Builder si la session est inexistente.
spark=SparkSession.builder.master('local').appName('house_price_pred').getOrCreate()

In [4]:
train_df = spark.read.csv('train.csv',header=True,inferSchema=True,nullValue="NA")
test_df  = spark.read.csv('test.csv',header=True,inferSchema=True,nullValue="NA")

In [5]:
from pyspark.sql.functions import isnull, when, count, col
null_columns=train_df.select([count(when(isnull(c), c)).alias(c) for c in train_df.columns])

In [6]:
null_columns.collect()

[Row(Id=0, MSSubClass=0, MSZoning=0, LotFrontage=259, LotArea=0, Street=0, Alley=1369, LotShape=0, LandContour=0, Utilities=0, LotConfig=0, LandSlope=0, Neighborhood=0, Condition1=0, Condition2=0, BldgType=0, HouseStyle=0, OverallQual=0, OverallCond=0, YearBuilt=0, YearRemodAdd=0, RoofStyle=0, RoofMatl=0, Exterior1st=0, Exterior2nd=0, MasVnrType=8, MasVnrArea=8, ExterQual=0, ExterCond=0, Foundation=0, BsmtQual=37, BsmtCond=37, BsmtExposure=38, BsmtFinType1=37, BsmtFinSF1=0, BsmtFinType2=38, BsmtFinSF2=0, BsmtUnfSF=0, TotalBsmtSF=0, Heating=0, HeatingQC=0, CentralAir=0, Electrical=1, 1stFlrSF=0, 2ndFlrSF=0, LowQualFinSF=0, GrLivArea=0, BsmtFullBath=0, BsmtHalfBath=0, FullBath=0, HalfBath=0, BedroomAbvGr=0, KitchenAbvGr=0, KitchenQual=0, TotRmsAbvGrd=0, Functional=0, Fireplaces=0, FireplaceQu=690, GarageType=81, GarageYrBlt=81, GarageFinish=81, GarageCars=0, GarageArea=0, GarageQual=81, GarageCond=81, PavedDrive=0, WoodDeckSF=0, OpenPorchSF=0, EnclosedPorch=0, 3SsnPorch=0, ScreenPorch=0,

In [7]:
#supprimer les colonne qui ont un taux de valeur null <NA> elevé
#Alley=1369, FireplaceQu=690, PoolQC=1453, Fence=1179, MiscFeature=1406
high_null_cols = ['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']
train_df=train_df.drop(*high_null_cols)
test_df=test_df.drop(*high_null_cols)

In [8]:
#remplacer les valeurs nulles des entiers par la moyennes de colonnes
from pyspark.sql.functions import avg
def fill_with_mean(this_df):
    included=[item[0] for item in this_df.dtypes if item[1].startswith('int')]
    stats = this_df.agg(*(avg(c).alias(c) for c in this_df.columns if c in included))
    return this_df.na.fill(stats.first().asDict())

train_df=fill_with_mean(train_df)
test_df=fill_with_mean(test_df)


In [9]:
#remplacer les valeurs nulles des chaines de caracteres par la chaine de caractere X
#BsmtQual=37, BsmtCond=37, BsmtExposure=38, BsmtFinType1=37 BsmtFinType2=38, GarageType=81, GarageYrBlt=81, GarageFinish=81,GarageQual=81, GarageCond=81, Electrical=1,MasVnrType=8,
from pyspark.sql.functions import translate
def fill_with_string(this_df,replacement):
    cols=[item[0] for item in this_df.dtypes if item[1].startswith('string')]
    for col in cols:
        this_df=this_df.na.fill(replacement,col)
    return this_df

train_df=fill_with_string(train_df,'X')
test_df=fill_with_string(test_df,'X')
 


In [10]:
null_columns=train_df.select([count(when(isnull(c), c)).alias(c) for c in train_df.columns])
null_columns.collect()

[Row(Id=0, MSSubClass=0, MSZoning=0, LotFrontage=0, LotArea=0, Street=0, LotShape=0, LandContour=0, Utilities=0, LotConfig=0, LandSlope=0, Neighborhood=0, Condition1=0, Condition2=0, BldgType=0, HouseStyle=0, OverallQual=0, OverallCond=0, YearBuilt=0, YearRemodAdd=0, RoofStyle=0, RoofMatl=0, Exterior1st=0, Exterior2nd=0, MasVnrType=0, MasVnrArea=0, ExterQual=0, ExterCond=0, Foundation=0, BsmtQual=0, BsmtCond=0, BsmtExposure=0, BsmtFinType1=0, BsmtFinSF1=0, BsmtFinType2=0, BsmtFinSF2=0, BsmtUnfSF=0, TotalBsmtSF=0, Heating=0, HeatingQC=0, CentralAir=0, Electrical=0, 1stFlrSF=0, 2ndFlrSF=0, LowQualFinSF=0, GrLivArea=0, BsmtFullBath=0, BsmtHalfBath=0, FullBath=0, HalfBath=0, BedroomAbvGr=0, KitchenAbvGr=0, KitchenQual=0, TotRmsAbvGrd=0, Functional=0, Fireplaces=0, GarageType=0, GarageYrBlt=0, GarageFinish=0, GarageCars=0, GarageArea=0, GarageQual=0, GarageCond=0, PavedDrive=0, WoodDeckSF=0, OpenPorchSF=0, EnclosedPorch=0, 3SsnPorch=0, ScreenPorch=0, PoolArea=0, MiscVal=0, MoSold=0, YrSold=

In [11]:
null_columns=test_df.select([count(when(isnull(c), c)).alias(c) for c in test_df.columns])
null_columns.collect()

[Row(Id=0, MSSubClass=0, MSZoning=0, LotFrontage=0, LotArea=0, Street=0, LotShape=0, LandContour=0, Utilities=0, LotConfig=0, LandSlope=0, Neighborhood=0, Condition1=0, Condition2=0, BldgType=0, HouseStyle=0, OverallQual=0, OverallCond=0, YearBuilt=0, YearRemodAdd=0, RoofStyle=0, RoofMatl=0, Exterior1st=0, Exterior2nd=0, MasVnrType=0, MasVnrArea=0, ExterQual=0, ExterCond=0, Foundation=0, BsmtQual=0, BsmtCond=0, BsmtExposure=0, BsmtFinType1=0, BsmtFinSF1=0, BsmtFinType2=0, BsmtFinSF2=0, BsmtUnfSF=0, TotalBsmtSF=0, Heating=0, HeatingQC=0, CentralAir=0, Electrical=0, 1stFlrSF=0, 2ndFlrSF=0, LowQualFinSF=0, GrLivArea=0, BsmtFullBath=0, BsmtHalfBath=0, FullBath=0, HalfBath=0, BedroomAbvGr=0, KitchenAbvGr=0, KitchenQual=0, TotRmsAbvGrd=0, Functional=0, Fireplaces=0, GarageType=0, GarageYrBlt=0, GarageFinish=0, GarageCars=0, GarageArea=0, GarageQual=0, GarageCond=0, PavedDrive=0, WoodDeckSF=0, OpenPorchSF=0, EnclosedPorch=0, 3SsnPorch=0, ScreenPorch=0, PoolArea=0, MiscVal=0, MoSold=0, YrSold=

In [12]:
train_df.printSchema()


root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = false)
 |-- LotFrontage: integer (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = false)
 |-- LotShape: string (nullable = false)
 |-- LandContour: string (nullable = false)
 |-- Utilities: string (nullable = false)
 |-- LotConfig: string (nullable = false)
 |-- LandSlope: string (nullable = false)
 |-- Neighborhood: string (nullable = false)
 |-- Condition1: string (nullable = false)
 |-- Condition2: string (nullable = false)
 |-- BldgType: string (nullable = false)
 |-- HouseStyle: string (nullable = false)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = false)
 |-- RoofMatl: string (nullable = false)
 |-- Exterior1st: string (nullable = false)
 |-- Exterior2nd: string 

In [13]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

In [14]:
cols_to_onehot_ignore=['Id','SalePrice','LotFrontage','LotArea','OverallQual','OverallQual','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageCars','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','MoSold','YearBuilt','YearRemodAdd','GarageYrBlt','YrSold']
cols_to_onehot = train_df.select([c for c in train_df.columns if c not in cols_to_onehot_ignore]).columns

In [15]:

strings_used = [var for var in cols_to_onehot]

stage_string = [StringIndexer(inputCol= c, outputCol= c+"_string_encoded") for c in strings_used]
stage_one_hot = [OneHotEncoder(inputCol= c+"_string_encoded", outputCol= c+ "_one_hot") for c in strings_used]
ppl = Pipeline(stages= stage_string + stage_one_hot)
train_df = ppl.fit(train_df).transform(train_df)


In [16]:
cols_to_onehot_ignore_test=['Id','LotFrontage','LotArea','OverallQual','OverallQual','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageCars','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','MoSold','YearBuilt','YearRemodAdd','GarageYrBlt','YrSold']
cols_to_onehot_test = test_df.select([c for c in test_df.columns if c not in cols_to_onehot_ignore_test]).columns
strings_used_test = [var for var in cols_to_onehot_test]
stage_string_test = [StringIndexer(inputCol= c, outputCol= c+"_string_encoded") for c in strings_used_test]
stage_one_hot_test = [OneHotEncoder(inputCol= c+"_string_encoded", outputCol= c+ "_one_hot") for c in strings_used_test]
ppl_test = Pipeline(stages= stage_string_test + stage_one_hot_test)
test_df = ppl_test.fit(test_df).transform(test_df)

In [17]:
train_df

DataFrame[Id: int, MSSubClass: int, MSZoning: string, LotFrontage: int, LotArea: int, Street: string, LotShape: string, LandContour: string, Utilities: string, LotConfig: string, LandSlope: string, Neighborhood: string, Condition1: string, Condition2: string, BldgType: string, HouseStyle: string, OverallQual: int, OverallCond: int, YearBuilt: int, YearRemodAdd: int, RoofStyle: string, RoofMatl: string, Exterior1st: string, Exterior2nd: string, MasVnrType: string, MasVnrArea: int, ExterQual: string, ExterCond: string, Foundation: string, BsmtQual: string, BsmtCond: string, BsmtExposure: string, BsmtFinType1: string, BsmtFinSF1: int, BsmtFinType2: string, BsmtFinSF2: int, BsmtUnfSF: int, TotalBsmtSF: int, Heating: string, HeatingQC: string, CentralAir: string, Electrical: string, 1stFlrSF: int, 2ndFlrSF: int, LowQualFinSF: int, GrLivArea: int, BsmtFullBath: int, BsmtHalfBath: int, FullBath: int, HalfBath: int, BedroomAbvGr: int, KitchenAbvGr: int, KitchenQual: string, TotRmsAbvGrd: int, 

In [18]:
test_df

DataFrame[Id: int, MSSubClass: int, MSZoning: string, LotFrontage: int, LotArea: int, Street: string, LotShape: string, LandContour: string, Utilities: string, LotConfig: string, LandSlope: string, Neighborhood: string, Condition1: string, Condition2: string, BldgType: string, HouseStyle: string, OverallQual: int, OverallCond: int, YearBuilt: int, YearRemodAdd: int, RoofStyle: string, RoofMatl: string, Exterior1st: string, Exterior2nd: string, MasVnrType: string, MasVnrArea: int, ExterQual: string, ExterCond: string, Foundation: string, BsmtQual: string, BsmtCond: string, BsmtExposure: string, BsmtFinType1: string, BsmtFinSF1: int, BsmtFinType2: string, BsmtFinSF2: int, BsmtUnfSF: int, TotalBsmtSF: int, Heating: string, HeatingQC: string, CentralAir: string, Electrical: string, 1stFlrSF: int, 2ndFlrSF: int, LowQualFinSF: int, GrLivArea: int, BsmtFullBath: int, BsmtHalfBath: int, FullBath: int, HalfBath: int, BedroomAbvGr: int, KitchenAbvGr: int, KitchenQual: string, TotRmsAbvGrd: int, 

In [19]:
#from pyspark.sql.functions import col
#df = train_df.select(*(col(c).cast('float').alias(c) for c in train_df.columns))

In [20]:
from pyspark.ml.feature import VectorAssembler

features = [var + "_one_hot" for var in strings_used]+cols_to_onehot_ignore
features.remove('Id')
#features.remove('SalePrice')

assembler = VectorAssembler(inputCols = features, outputCol= "features")
output = assembler.transform(train_df)

In [21]:
datas=output.select('features','SalePrice')

In [22]:
from pyspark.ml.regression import LinearRegression 

In [23]:
reg=LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8,labelCol='SalePrice')

In [24]:
fited_model=reg.fit(datas)

In [25]:
training_sum=fited_model.summary

In [26]:
training_sum.predictions.describe().show()

+-------+------------------+------------------+
|summary|         SalePrice|        prediction|
+-------+------------------+------------------+
|  count|              1460|              1460|
|   mean|180921.19589041095|180921.19589041104|
| stddev| 79442.50288288663|  81120.5679501909|
|    min|           34900.0|30554.472208220628|
|    max|          755000.0| 720237.2740787979|
+-------+------------------+------------------+



In [27]:
#print("Coefficients: " + str(fited_model.coefficients))
print("Intercept: " + str(fited_model.intercept))

Intercept: 823344.806791


In [28]:
print("RMSE: %f" % training_sum.meanSquaredError)
print("r2: %f" % training_sum.r2)

RMSE: 155139759.191228
r2: 0.975401


In [33]:
features1 = [var + "_one_hot" for var in strings_used_test]+cols_to_onehot_ignore_test
features1.remove('Id')

assembler1 = VectorAssembler(inputCols = features1, outputCol= "features")
output1 = assembler1.transform(test_df)
datas1=output1.select('Id','features')

In [34]:
lr_predictions = fited_model.transform(output)
lr_predictions.select("Id","features","SalePrice").show(5)


+---+--------------------+---------+
| Id|            features|SalePrice|
+---+--------------------+---------+
|  1|(265,[1,14,18,19,...|   208500|
|  2|(265,[0,14,18,19,...|   181500|
|  3|(265,[1,14,18,20,...|   223500|
|  4|(265,[6,14,18,20,...|   140000|
|  5|(265,[1,14,18,20,...|   250000|
+---+--------------------+---------+
only showing top 5 rows



In [31]:
features1

['MSSubClass_one_hot',
 'MSZoning_one_hot',
 'Street_one_hot',
 'LotShape_one_hot',
 'LandContour_one_hot',
 'Utilities_one_hot',
 'LotConfig_one_hot',
 'LandSlope_one_hot',
 'Neighborhood_one_hot',
 'Condition1_one_hot',
 'Condition2_one_hot',
 'BldgType_one_hot',
 'HouseStyle_one_hot',
 'OverallCond_one_hot',
 'RoofStyle_one_hot',
 'RoofMatl_one_hot',
 'Exterior1st_one_hot',
 'Exterior2nd_one_hot',
 'MasVnrType_one_hot',
 'ExterQual_one_hot',
 'ExterCond_one_hot',
 'Foundation_one_hot',
 'BsmtQual_one_hot',
 'BsmtCond_one_hot',
 'BsmtExposure_one_hot',
 'BsmtFinType1_one_hot',
 'BsmtFinType2_one_hot',
 'Heating_one_hot',
 'HeatingQC_one_hot',
 'CentralAir_one_hot',
 'Electrical_one_hot',
 'KitchenQual_one_hot',
 'Functional_one_hot',
 'GarageType_one_hot',
 'GarageFinish_one_hot',
 'GarageQual_one_hot',
 'GarageCond_one_hot',
 'PavedDrive_one_hot',
 'SaleType_one_hot',
 'SaleCondition_one_hot',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallQual',
 'MasVnrArea',
 'BsmtFinSF1',
